In [1]:
import pandas as pd

import re
import pickle
from pathlib import Path

import datasets

import nltk
from nltk.tokenize import sent_tokenize

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/egor-work/mambaforge/envs/nlp_train/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset('c00k1ez/summarization')

Found cached dataset parquet (/Users/egor-work/.cache/huggingface/datasets/c00k1ez___parquet/c00k1ez--summarization-7ccb6690da50a6b9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 399.86it/s]


In [3]:
train_part = dataset['train']
train_part[0]

{'chapter_id': 406,
 'book_id': 97,
 'chapter_title': 'Глава 1.Смерть отца. Переезд в Нижний Новгород. Знакомство с дедом',
 'chapter_summary': 'Первое воспоминание Алёши — смерть отца. Он не понимал, что отца больше нет, но запомнил плач Варвары. Варвара — мама Алёши, вдова, ещё молода и красива, крупная, сильная, с густыми светлыми волосами, непокорная, с сильным характером. Помочь с похоронами и сильно заболевшим Алёшей приехала бабушка Акулина. Акули́на Ивановна Каши́рина — бабушка Алёши, крупная, полная, большеголовая, с огромными глазами, рыхлым носом и длинными чёрными густыми волосами, очень добрая, мудрая и сильная, знает много сказок. В день смерти мужа у Варвары начались преждевременные роды, ребёнок родился слабым. После похорон бабушка забрала всех в Нижний Новгород. На пароходе малыш умер. Сначала бабушка показалась Алёше странной и смешной. Говорила она, особенным образом выпевая слова. Бабушка стала для Алёши «самым близким сердцу... дорогим человеком». Путешествие по В

In [4]:
chapters_text = list(train_part['chapters_text'])
chapter_summary = list(train_part['chapter_summary'])

In [5]:
chapters_text_len = []
summary_len = []

for text, summ in zip(chapters_text, chapter_summary):
    chapters_sentences = sent_tokenize(text, language='russian')
    chapters_text_len.append(len(chapters_sentences))
    summ_sentences = sent_tokenize(summ, language='russian')
    summary_len.append(len(summ_sentences))

In [6]:
chapters_text_len = np.array(chapters_text_len)
summary_len = np.array(summary_len)

ratio = summary_len / chapters_text_len

In [7]:
ratio.mean()

0.16223097910775255

In [8]:
from src.text_rank import TextRank

[nltk_data] Downloading package punkt to /Users/egor-work/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
tr = TextRank()

t = tr(train_part[0]['chapters_text'], ratio=ratio.mean())

In [15]:
t

[{'id': 2,
  'score': 0.007742397776552917,
  'sentence': 'Меня держит за руку бабушка, — круглая, большеголовая, с огромными глазами и смешным рыхлым носом; она вся черная, мягкая и удивительно интересная; она тоже плачет, как-то особенно и хорошо подпевая матери, дрожит вся и дергает меня, толкая к отцу; я упираюсь, прячусь за нее; мне боязно и неловко.'},
 {'id': 15,
  'score': 0.008209396555632536,
  'sentence': 'Я с первого же дня подружился с нею, и теперь мне хочется, чтобы она скорее ушла со мною из этой комнаты.'},
 {'id': 17,
  'score': 0.00786153912575226,
  'sentence': 'Я впервые вижу ее такою, — она была всегда строгая, говорила мало; она чистая, гладкая и большая, как лошадь; у нее жесткое тело и страшно сильные руки.'},
 {'id': 18,
  'score': 0.007371024430062553,
  'sentence': 'А сейчас она вся как-то неприятно вспухла и растрепана, всё на ней разорвалось; волосы, лежавшие на голове аккуратно, большою светлой шапкой, рассыпались по голому плечу, упали на лицо, а половин